In [14]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Our processor

In [13]:
! lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   24
  On-line CPU(s) list:    0-23
Vendor ID:                GenuineIntel
  Model name:             13th Gen Intel(R) Core(TM) i7-13700
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   16
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   23%
    CPU max MHz:          5200.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4224.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc c

# Choose the "right" metrics

## Throughput and Latency

### SSD v.s. HDD

You may use to hdparm (need root permission to execute). The /dev/sda on this machine is an SATA SSD that has around 450-500MB/sec bandwidth. The /dev/md0 is a RAID contains two H.D.Ds in RAID-0 configuration that also achieves 450-500MB/sec bandwidth. Let's examine the bandwidth using the following command.

In [ ]:
from IPython.display import IFrame
IFrame("https://hub.escalab.org:8000/user/htseng/terminals/1", width="100%", height="400")

In [ ]:
! cp ../300000000.bin /tmp/300000000.bin
! echo "Configuration,Size,ET,FileInput,CPU_Kernel,GPU_Kernel,Host2GPU,GPU2Host" > sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 512 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 512 2>> ../sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 32768 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 32768 2>> ../sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 262144 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 262144 2>> ../sort_small.csv
rm -f /tmp/300000000.bin
display_df_mono((render_csv("sort_small.csv")))

What can we observe here?

## FLOPs

In [15]:
! cd metrics; make; echo "size,GLOPS,GPU_time" > flops.csv

make: Nothing to be done for 'all'.


In [16]:
! cd ./metrics; time ./cpumm 2048 512

Data type size: 4

Time: 4394.022 ms

Throughput: 3.91 GFLOPS


real	0m4.495s
user	0m4.478s
sys	0m0.016s


In [20]:
! cd ./metrics; time ./cudamm 2048 1 2>> flops.csv 

Data Type Size: 4
Time elapsed on matrix multiplication of 2048x2048 . 2048x2048 on GPU: 20.753952 ms.


Throughput: 827.79 GFLOPS


real	0m0.447s
user	0m0.116s
sys	0m0.325s


In [25]:
! cd ./metrics; time ./cudamm 4096 1 2>> flops.csv 

Data Type Size: 4
Time elapsed on matrix multiplication of 4096x4096 . 4096x4096 on GPU: 173.070023 ms.


Throughput: 794.12 GFLOPS


real	0m0.836s
user	0m0.497s
sys	0m0.334s


In [26]:
! cd metrics; time ./cudamm 8192 1 2>> flops.csv 

Data Type Size: 4
Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 816.980469 ms.


Throughput: 1345.82 GFLOPS


real	0m2.653s
user	0m2.121s
sys	0m0.523s


In [27]:
! ssh htseng@lapras "nvidia-smi; time /nfshome/htseng/courses/CSE142/demo/amdahlslaw/metrics/cudamm 8192 1 2>> /nfshome/htseng/courses/CSE142/demo/amdahlslaw/metrics/flops.csv "

Tue Aug  6 15:10:59 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0 Off |                  Off |
|  0%   45C    P8             14W /  450W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
FLOPS_Ratio = 781.863586/348.333954
print(FLOPS_Ratio)

In [ ]:
Speedup = 2.610/1.991
print(Speedup)

In [ ]:
! cd metrics; time ./cudamm_double 2048 1

In [ ]:
! cd metrics;  ./cudamm_double 2048 0

In [ ]:
! cd metrics; time ./cpumm_double 2048 512

In [ ]:
! cd metrics;  time ./cudamm 2048 0